In [1]:
# Imports
import torch
from torch.distributions import Normal,Gamma,Laplace,StudentT, Bernoulli
from dask_jobqueue import SLURMCluster
from distributed import Client
import CSuite_SCM

In [2]:
# Args set up
ntrial = 20
datasets = ["Fork","Simpson","NonlinG"]
dists = [Normal(0,1),Bernoulli(0.5),Gamma(1,1)]
transforms = [lambda x : x, lambda x : 2*x - 1, lambda x : 1 / x - 1]
num_change = [0,4,4]
model_names = ["G-RQS SCM","L-RQS SCM","TA-RQS SCM"]
model_base_dists = [Normal(0,1),Laplace(0,1),StudentT(10,0,1)]
use_RQS_flow = [True,True,True]
RQS_bins = [8,8,8]
tail_adapt = [False,False,True]
batch_size = 64
scheduler = False
decay = 0

In [3]:
cluster = SLURMCluster(
    n_workers=0,
    memory="16GB",
    processes=1,
    cores=1,
    scheduler_options={
        "dashboard_address": ":10000",
        "allowed_failures": 10
    },
    job_cpu=1,
    walltime="48:0:0",
)
cluster.adapt(minimum=0, maximum=200)
client = Client(cluster)

futures = []
for seed in range(ntrial):
    for d in range(len(datasets)):
        for j in range(len(dists)):
            for m in range(len(model_names)):
                f = client.submit(CSuite_SCM.run_experiment,seed,
                                                              datasets[d],
                                                              dists[j],
                                                              transforms[j],
                                                              num_change[j],
                                                              model_names[m],
                                                              model_base_dists[m],
                                                              use_RQS_flow[m],
                                                              RQS_bins[m],
                                                              tail_adapt[m],
                                                              batch_size,
                                                              scheduler,
                                                              decay)
                futures.append(f)

In [4]:
futures

[<Future: pending, key: run_experiment-a85d02e0148fd7dffcc98b87ea6295f2>,
 <Future: pending, key: run_experiment-6b1ad5c929b872c84134ddc496900f8d>,
 <Future: pending, key: run_experiment-e2ec65225ec8394755f1d08426e07a96>,
 <Future: pending, key: run_experiment-4dec8b8324cdd6411dbaec14d2f53dcf>,
 <Future: pending, key: run_experiment-7bf08c7711f3aef14a99e707d7929d38>,
 <Future: pending, key: run_experiment-722bd51116689764a0e4f01592b917f6>,
 <Future: pending, key: run_experiment-98603aa07d9340e7e0deb25143902183>,
 <Future: pending, key: run_experiment-26bc0b4942c08183491fc57b093e6dd4>,
 <Future: pending, key: run_experiment-14f009c344aae1b87e4c7a8f544d84e4>,
 <Future: pending, key: run_experiment-89093be265e982fc48b4cbb89d0875f9>,
 <Future: pending, key: run_experiment-4bf82bd64d052d47a85a60b96cae99e5>,
 <Future: pending, key: run_experiment-6384d1faa51b798b4ce3f49afccc5be6>,
 <Future: pending, key: run_experiment-15bbfff7b0569205574b82d1db44a09f>,
 <Future: pending, key: run_experiment

In [5]:
results = client.gather(futures)

In [6]:
client.close()
cluster.close()

2025-03-19 17:04:49,684 - distributed.deploy.adaptive_core - INFO - Adaptive stop


In [7]:
torch.save(f = "Csuite_SCM_results_trials={0}_scheduler={1}_decay={2}_CV.pt".format(ntrial,scheduler,decay), obj = results)